In [ ]:
def plot(x, p=None, labels=False):
    plt.figure(figsize=(20,2))
    for i in range(10): # refers to the first 10 images that we are plotting
        plt.subplot(1, 10, i+1) # 1 row and 10 columns; the index for the subplot 
        plt.imshow(x[i].reshape(28, 28), # this will show the image except we will have to reshape to 28 by 28 because we flattened it in the previous task.
                   cmap = 'binary') # "binary" so that we see the black and white images as they are.
        plt.xticks([])
        plt.yticks([])
        if labels: # if labels is true,
            plt.xlabel(np.argmax(p[i])) # then we also want to label our X axis.
    plt.show()
    return

In [ ]:
import numpy as np
from tensorflow.keras.datasets import mnist
from matplotlib import pyplot as plt
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.callbacks import EarlyStopping, LambdaCallback
from tensorflow.keras.utils import to_categorical
import random
import cv2
import os
%matplotlib inline

# Stap 1: importeer de images
#### - De afbeeldingen met ruis
#### - De afbeeldingen zonder ruis
Zorg dat de afbeeldingen:
- In greyscale staan
- De afbeeldingen genormaliseerd zijn (0-1)
- De shape (len(dataset),28,28) is
- Tip, je kan de dataset visualiseren met de plot functie, hierboven gedefinieerd

# Stap 2: Voeg noise toe
- Voeg ruis toe aan de afbeeldingen, zoek zelf uit hoe je dit moet doen
- kijk of het goed is gegaan door de ruis te vergelijken met de gekregen afbeeldingen met ruis

# Stap 3: Train de autoencoder
- Hieronder heb ik een model gedefinieerd die jullie gaan gebruiken
- Voordat je het model kan trainen moet je de shape van de dataset in 2d zetten, gebruik hiervoor de reshape functie van numpy
- De nieuwe shape moet (len(dataset), 784) zijn
- Train vervolgens de dataset

In [ ]:
def train_autoencoder(X_train_noisy, X_train):
    """
    Deze functie heeft 2 datasets nodig, de afbeeldingen met ruis en die zonder ruis,
    De functie geeft de getrainde autoencoder, en print de training loss.
    
    de .predict methode kan gebruikt worden om de predictions te krijgen
    
    """
    input_image = Input(shape=(784,))
    encoded = Dense(64, activation='relu')(input_image)
    decoded = Dense(784, activation='sigmoid')(encoded)
    autoencoder = Model(input_image, decoded)
    autoencoder.compile(loss='binary_crossentropy', optimizer='adam')
    autoencoder.fit(
        X_train_noisy, X_train, 
        epochs=100,
        batch_size=512, 
        validation_split=0.2, # Use a validation split of 20%,
        verbose=False, # set verbose to false because we don't want to actually use any build logs. 
        callbacks=[
            EarlyStopping(monitor='val_loss', patience=5),
            LambdaCallback(on_epoch_end=lambda e,l: print('{:.4f}'.format(l['val_loss']), end=' _ '))
        ]
    )
    print(' _ ')
    print('Training is complete!')
    return autoencoder

# Stap 3: Predictions
- Haal nu de ruis weg bij de gekregen afbeeldingen met ruis
- Zet eerst de afbeeldingen om naar 2d
- Stop ze daarna in het model
- Zet daarna de output weer terug naar 3d
- kijk of de ruis goed is weggehaald